# GENIE3

In [1]:
from scprint import scPrint
from scdataloader import Preprocessor

from bengrn import BenGRN, get_perturb_gt

from bengrn.base import train_classifier

from bengrn import compute_genie3
from grnndata import utils as grnutils
from anndata.utils import make_index_unique
import pandas as pd
import numpy as np

import scanpy as sc

%load_ext autoreload
%autoreload 2 

import torch
torch.set_float32_matmul_precision('medium')

→ connected lamindb: jkobject/scprint


In [2]:
adata = get_perturb_gt()


In [0]:
from bengrn import BenGRN, get_perturb_gt
from bengrn import compute_genie3
from grnndata import utils as grnutils
import scanpy as sc
%load_ext autoreload
%autoreload 2 
import torch
torch.set_float32_matmul_precision('medium')
adata = get_perturb_gt()
adata.var["isTF"] = False
adata.var.loc[adata.var.gene_name.isin(grnutils.TF), "isTF"] = True
adata.var["isTF"].sum()
genes = torch.load('/pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/scprint_scale/o2uniqsx/checkpoints/epoch=18-step=133000.ckpt', map_location=torch.device('cpu'))['hyper_parameters']['genes']
adata.var['symbol'] = adata.var["gene_name"]
adata.var['ensembl_id'] = adata.var.index
subadata = adata[:, adata.var.index.isin(
    genes)]
del subadata.varp['GRN']
sc.pp.normalize_total(subadata)
sc.pp.log1p(subadata)
genie_grn = compute_genie3(subadata, nthreads=64, ntrees=1000)
genie_grn.var.index = subadata.var.index
BenGRN(genie_grn, do_auc=True, doplot=True).compare_to(other=adata)

ModuleNotFoundError: No module named 'bengrn'

In [3]:
genes = torch.load('/pasteur/zeus/projets/p02/ml4ig_hot/Users/jkalfon/scprint_scale/o2uniqsx/checkpoints/epoch=18-step=133000.ckpt', map_location=torch.device('cpu'))['hyper_parameters']['genes']

#genes = torch.load('../data/temp/vbd8bavn/epoch=17-step=90000.ckpt')['hyper_parameters']['genes']

In [4]:
preprocessor = Preprocessor(force_preprocess=True, skip_validate=True,
                            do_postp=False, min_valid_genes_id=5000, min_dataset_size=64)
nadata = preprocessor(adata.copy())
NUM_GENES = 5000
NUM_CELLS = 1024
adata

Dropping layers:  KeysView(Layers with keys: )
checking raw counts


removed 0 non primary cells, 10691 renamining


filtered out 0 cells, 10691 renamining
Removed 1 genes.


startin QC


Seeing 10066 outliers (94.15% of total dataset):


done


GRnnData object with n_obs × n_vars = 10691 × 8563
    obs: 'gem_group', 'gene', 'gene_id', 'transcript', 'gene_transcript', 'sgID_AB', 'mitopercent', 'UMI_count', 'z_gemgroup_UMI', 'core_scale_factor', 'core_adjusted_UMI_count', 'organism_ontology_term_id'
    var: 'gene_name', 'chr', 'start', 'end', 'class', 'strand', 'length', 'in_matrix', 'mean', 'std', 'cv', 'fano', 'isTF'
    varp: 'GRN'
    with a grn of 1115065 elements

In [5]:
sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=NUM_GENES)
adata.var['symbol'] = adata.var["gene_name"]
adata.var['ensembl_id'] = adata.var.index
subadata = adata[:, adata.var.index.isin(
    genes)]
del subadata.varp['GRN']
sc.pp.normalize_total(subadata)
sc.pp.log1p(subadata)

In [6]:
genie_grn = compute_genie3(subadata, nthreads=64, ntrees=1000)
genie_grn.var.index = subadata.var.index
BenGRN(genie_grn, do_auc=True, doplot=True).compare_to(other=adata)

Tree method: RF
K: sqrt
Number of trees: 1000


running jobs on 64 threads


In [ ]:
genie_grn.var.index = make_index_unique(genie_grn.var['symbol'].astype(str))
BenGRN(genie_grn, do_auc=True, doplot=True).scprint_benchmark()